In [260]:
###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# 🏭 Power Plant ON/OFF: Predictions

* **Time estimate**: 1 hour
* **Cost estimate**: less than **5**.00 USD

This is an **interactive** notebook that contains **all** of the **code** necessary to train an ML model from satellite images for a geospatial classification of whether a power plant is on/off. 

This is a first step **introductory example** of how this **satellite images** can be used to detect **carbon pollution** from power plants.

--------------------------------------------


💚 This is one of many **machine learning how-to samples** inspired from **real climate solutions** aired on the [People and Planet AI 🎥 series](https://www.youtube.com/playlist?list=PLIivdWyY5sqI-llB35Dcb187ZG155Rs_7).

# 🚏 Overview

This notebook leverages geospatial data from [Google Earth Engine](https://earthengine.google.com/) and labeled data provided by the organization [Climate TRACE](https://www.climatetrace.org/). By combining these two data sources, you'll build and train a model that predicts whether or not a power plant is turned on and producing emissions.

### 🛰️ Data _(inputs)_

The data in this example consists of images from a satellite called [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#description), for a wide-swath, **high-resolution**, multi-spectral imaging for land monitoring studies.

When working with satellite data, each input image has the **dimensions** `[width, height, bands]`. **Bands** are measurements from specific satellite instruments for different ranges of the **electromagnetic spectrum**. For example, Sentinel-2 contains [🌈 13 spectral bands](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#bands). If you are familiar with image classification problems, you can think of the bands as similar to an image's RGB (red, green, blue) channels. However, when working with satellite data we generally have **more than just 3** channels.

![satellite_inputs](img/inputs.png)



### 🏷️ Labels _(outputs)_

For each patch of pixels (an image of a power plant) that we give to the model, it performs **binary classification**, which indicates whether the power plant is on or off.

In this example, the **output** is a single number between *0 (Off) and 1 (On)*, representing the **probability** of that power plant being ON.

![satellite_outputs](img/outputs.png)


### Model _(function)_

**TL;DR**
*The model will receive a patch of pixels, in the center is the power plant tower. We then take 16 pixels as padding creating a 33x33 patch. This returns a classification of ON/OFF*

In this example, we have a CSV file of labels. Each row in this file represents a power plant at a specific lat/lon and timestamp. To deal with the sparsity, at training time we'll prepare a dataset where each input image is a single pixel that we have a label for. We will then add a padding around that image. These padded pixels will not get predictions, but will help our model to make better predictions for the center point that we have a label for.

For example, with a padding of 16, each 1 pixel input point would become a 33x33 image after the padding is added.

![training](img/training.png)


At prediction time, we can pass in an image of any size. We'll add a layer of padding around the image, and will get predictions for each pixel except for the padding.

To handle the variable input size, we'll use a fully convolutional neural network. Using this type of architecture will allow us to have variable input image sizes. Note that while the height and width of the image can be variable, the number of bands must be constant.

## 🚴‍♀️ Steps summary

Here's a quick summary of what you’ll go through:

1. **Get the training data**:
  Extract satellite images from [Earth Engine](https://earthengine.google.com/), combine it with the data that was labeled and contains lat/long coordinates from [Climate TRACE](https://climatetrace.org) in a CSV, and export to
  [Cloud Storage](https://cloud.google.com/storage).

1. **Run a custom training job**:
  Using [Tensorflow Keras](https://keras.io/) on [Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training) using a [pre-built training container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers).

1. **Deploy a web service to host the trained model**:
  On
  [Cloud Run](https://cloud.google.com/run)
  and get predictions using the model.

1. **Get Predictions**:
  Use the web service to get predictions for new data.

1. **Visualize predictions**:
  Visualize the preductions on a map.

1. (Optional) **Delete the project** to avoid ongoing costs.

## 🙈 Using this interactive notebook

Click the **run** icons ▶️ of each section within this notebook. 

This notebook code lets you train and deploy an ML model, as well as test its accuracy, from end-to-end. When you run a code cell, the code runs in the notebook's runtime, so you're not making any changes to your personal computer.

> 🛎️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.
You can use an existing project and the cost will be less than **$5.00**. Alternatively, you can create a new Cloud project with cloud credits for free

## ✨ Before you begin, you need to…

1. Decide on creating a new
   [free project](https://cloud.google.com/free/docs/gcp-free-tier)
   _(recommended)_ or using an existing one.
   Then **copy the project ID** and paste it in the `google_cloud_project` field in the "Entering project details” section below.

   > 💡 If you _don't plan to keep the resources_ that you create via this sample, we recommend creating a new project instead of selecting an existing project.
   > After you finish these steps, you can delete the project, removing all the resources associated in bulk.

1. [_Click here_](https://console.cloud.google.com/flows/enableapi?apiid=dataflow,aiplatform.googleapis.com)
   to **enable the following APIs** in your Google Cloud project:
   _Dataflow_ and _AI Platform_.

1. Make sure that **billing is enabled** for your Google Cloud project,
   [_click here_](https://cloud.google.com/billing/docs/how-to/modify-project)
   to learn how to confirm that billing is enabled.

1. Have an **Earth Engine** account (it's FREE) or create a new one.
  To create an account, fill out the [registration form here.](https://signup.earthengine.google.com/#!/). Please note this can take from 0-24 hours...but it's worth it! Come back to this sample after you have this.

### ⛏️ Preparing the project environment

Click the run ▶️ icons in order for the cells to download and install the necessary code, libraries, and resources for this solution.

> 💡 You can _optionally_ view the entire
> [code in GitHub](https://github.com/GoogleCloudPlatform/python-docs-samples/tree/main/people-and-planet-ai/geospatial-classification).

### ↘️ Install the Vertex AI SDK

Note that you will be prompted to restart the runtime after installation.

In [ ]:
!pip3 install --user --upgrade google-cloud-aiplatform

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=project, staging_bucket=bucket)

### ↘️ Get the code

In [332]:
# Get the sample source code.

## TODO

### ✏️ Enter your Cloud project's details. Ensure you provide a regional bucket!

In [ ]:
#@title My Google Cloud resources
project = '' #@param {type:"string"}
bucket = '' #@param {type:"string"}
region = 'us-central1' #@param {type:"string"}

# Validate the inputs.
if not project:
  raise ValueError(f"Please provide a value for 'project'")
if not bucket:
  raise ValueError(f"Please provide a value for 'bucket'")
if not region:
  raise ValueError(f"Please provide a value for 'region'")

# Authenticate
from google.colab import auth

auth.authenticate_user()
print('Authenticated')

!gcloud config set project {project}

### 🗺️ Authenticate to Earth Engine

In order to use the Earth Engine API, you'll **need** to have an **Earth Engine account**.

To create an account, fill out the [registration form here.](https://signup.earthengine.google.com/#!/)

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

# 1. 🛰️ Get the training data

The training data in this sample comes from **two places**: 

1. The satellite images will be extracted from *Earth Engine*.

2. The **labels** are provided in a *CSV file* that indicates whether a *power plant* is turned *on or off* at a **particular timestamp**. 

For each row in the CSV file, we need to extract the corresponding Sentinel image taken at that specific latitude/longitude and timestamp. We'll **export** this image data, along with the corresponding label (on/off), to Cloud Storage.

In [3]:
# Import libraries

import pandas as pd
import numpy as np
from pprint import pprint

In [109]:
# Define constants

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
LABEL = 'is_powered_on'

### 🏷️ Import labels

First, we **import** the **CSV** file that contains the labels.

In [584]:
labels_dataframe = pd.read_csv('labeled_geospatial_data.csv')

Each row in this dataframe represents a power plant at a particular timestamp. 

The label column indicates whether or not the power plant was turned **on (1)** or **off (0)** at that timestamp.

In [ ]:
labels_dataframe.head()

Next, we import this data into an Earth Engine [`Feature Collection`](https://developers.google.com/earth-engine/apidocs/ee-featurecollection).

In Earth Engine, a [`Feature`](https://developers.google.com/earth-engine/guides/features) is an object with a _geometry property_ storing a [`Geometry`](https://developers.google.com/earth-engine/guides/geometries) object, and a _properties property_ storing a dictionary of other properties. Groups of related `Features` can be combined into a `FeatureCollection` to enable additional operations on the entire set such as filtering, sorting, and rendering.

In [659]:
def create_label_fc(path):
  '''Creates a FeatureCollection from the label dataframe.'''

  dataframe = pd.read_csv(path)
  num_examples = dataframe.shape[0]
  data_dict = dataframe.to_dict()
  feats = []
  properties = ['timestamp', 'is_powered_on']
  for idx in np.arange(num_examples):
    feat_dict = {}
    geometry = ee.Geometry.Point([data_dict['lon'][idx], data_dict['lat'][idx]])  
    for feature in properties:
      feat_dict[feature] = data_dict[feature][idx]
    feat = ee.Feature(geometry, feat_dict)
    feats.append(feat)
  return ee.FeatureCollection(feats)

In [660]:
label_fc = create_label_fc('labeled_geospatial_data.csv')

Let's take a look at the first element in the `FeatureCollection`. You can see that this is equivalent to the first element in our dataframe, but just in a different format.

In [662]:
pprint(label_fc.first().getInfo())

{'geometry': {'coordinates': [-84.80529, 39.11613], 'type': 'Point'},
 'id': '0',
 'properties': {'label': 1, 'timestamp': '2020-07-03 16:32:41.397000+00:00'},
 'type': 'Feature'}


Next, we define a preprocessing function that will reformat the timestamp into two new properties: `start` and `end`.

In [663]:
def format_date(feature):
  '''Creates start date and end date properties.'''
    
  # Extract start and end dates
  timestamp = ee.String(feature.get('timestamp')).split(' ').get(0)
  year =  ee.Number.parse(ee.String(timestamp).split('-').get(0))
  month = ee.Number.parse(ee.String(timestamp).split('-').get(1))
  day =   ee.Number.parse(ee.String(timestamp).split('-').get(2))
  start = ee.Date.fromYMD(year, month, day)
  end = start.advance(1, 'day')

  # Create new feature
  feature = feature.set({'start': start, 'end': end})

  return feature

We map this preprocessing function across our `FeatureCollection`.

In [664]:
label_fc = label_fc.map(format_date)

Let's examine the first feature in this collection to see what it looks like after the map operation. 

You can see that each feature has a `Geometry`, which is a specific latitude and longitude, as well as a set of properties:

* `label` indicates whether the power plant is on (1) or off (0)
* `start` is the datetime for when this observation was made
* `end` is a datetime one day after `start`
* `timestamp` is the original timestamp pulled from the CSV file

The `label` property is what we want our model to predict. And the `start` and `end` timestamps will be helpful when we extract the corresponding Sentinel images because each label is for a specific point in time.

In [665]:
pprint(label_fc.first().getInfo())

{'geometry': {'coordinates': [-84.80529, 39.11613], 'type': 'Point'},
 'id': '0',
 'properties': {'end': {'type': 'Date', 'value': 1593820800000},
                'label': 1,
                'start': {'type': 'Date', 'value': 1593734400000},
                'timestamp': '2020-07-03 16:32:41.397000+00:00'},
 'type': 'Feature'}


### Merge 🏷️ labels + 🛰️ Sentinel image data

In Earth Engine, an [`ImageCollection`](https://developers.google.com/earth-engine/guides/ic_creating) is a stack or sequence of images. An [`Image`](https://developers.google.com/earth-engine/guides/image_overview) is composed of one or more bands and each band has its own name, data type, scale, mask and projection. The [`Sentinel-2`](https://developers.google.com/earth-engine/guides/ic_creating) dataset is represented as an `ImageCollection`, where each image in the collection is of a specific geographic location at a particular time.

In the cell below, we write a function to extract the Sentinel image taken at the specific latitude/longitude and timestamp for each `Feature` in our `FeatureCollection`. 

To do this, we first filter the Sentinel-2 `ImageCollection` at the start/end dates for a particular `Feature`.

Then, using the [`neighorboodToArray`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray) method we create a new `FeatureCollection` that contains the satellite data for each band at the latitude and longitude of interest as well as a 16 pixel padding around that point.

In the image below you can think of the purple box representing the lat/lon where the power plant is located. And around this pixel, we add the padding.

![training](img/training.png)

In [592]:
def get_neighboring_patch(feature):
  '''Gets image pixel values for patch.'''
    
  # filter ImageCollection at start/end dates.   
  image = ee.ImageCollection('COPERNICUS/S2').filterDate(
      feature.get('start'), feature.get('end')).select(BANDS).median()
  
  # extract pixel values at the lat/lon with a 16x16 padding
  return ee.FeatureCollection([
      image.neighborhoodToArray(ee.Kernel.square(16)).sampleRegions(
          collection=ee.FeatureCollection([feature]), scale=10).first()
  ])

We map this function across the `FeatureCollection` and flatten it so we have a single `FeatureCollection`.

In [593]:
data = label_fc.map(get_neighboring_patch).flatten()

Let's look at the properties for the first element in this new `FeatureCollection`. You can see that it still contains the `start`, `end`, `label`, and `timestamp` properties, but with  has 13 additional properies, one for each spectral band.

In [594]:
data.first().propertyNames().getInfo()

['system:index',
 'start',
 'end',
 'label',
 'timestamp',
 'B10',
 'B11',
 'B12',
 'B8A',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9']

The data contained in each band property is an array of shape 33x33.

For example, here is the data for band B1 in the first element of our `FeatureCollection` expressed as a numpy array.

In [595]:
example_feature = np.array(data.first().get('B1').getInfo())
print(example_feature)
print('shape: ' + str(example_feature.shape))

[[1803 1803 1803 ... 1797 1797 1797]
 [1803 1803 1803 ... 1797 1797 1797]
 [1803 1803 1803 ... 1797 1797 1797]
 ...
 [1519 1519 1519 ... 1560 1560 1560]
 [1519 1519 1519 ... 1560 1560 1560]
 [1519 1519 1519 ... 1560 1560 1560]]
shape: (33, 33)


### 🎛️ Create train/validation splits

Before we can train an ML model, we need to split this data into training and validation datasets. We can easily do this by adding a random column to our `FeatureCollection` and then filtering the collection on that column.

In [623]:
data = data.randomColumn()

In [296]:
# 70% of data for training and 30% for validation

training = data.filter(ee.Filter.lt('random', 0.7))
validation = data.filter(ee.Filter.gte('random', 0.7))

### 💾 Export data

Lastly, we'll export the data to a Cloud Storage bucket. We'll export the data as [TFRecords](https://www.tensorflow.org/tutorials/load_data/tfrecord) and only export the bands and label properties.

Later when we run the training job, we'll parse these TFRecords and feed them to the model.

In [602]:
# Export data

training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description="Training image export",
  bucket=bucket,
  fileNamePrefix="geospatial_training",
  selectors= BANDS + [LABEL],
  fileFormat='TFRecord')

training_task.start()

validation_task = ee.batch.Export.table.toCloudStorage(
  collection=validation,
  description="Validation image export",
  bucket=bucket,
  fileNamePrefix="geospatial_validation",
  selectors= BANDS + [LABEL],
  fileFormat='TFRecord')

validation_task.start()

This export will take around 35 minutes. You can monitor the progress with the following command:

In [4]:
pprint(ee.batch.Task.list())

# 2. 👟 Run a custom training job

Once the export jobs have finished, we're **ready to use** that data to train a model on Vertex AI Training.

The complete training code can be found in the `task.py` file.

To run our custom training job on Vertex AI Training, we'll use the [pre-built containers](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) provided by Vertex AI to run our training script.


In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="geospatial_model_training",
    script_path="task.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest")

The job will take around 10 minutes to run.

In [ ]:
model = job.run(args=[f'--bucket={bucket}'])

# 3. 💻Deploy a web service to host the trained model

Next, we use
[Cloud Run](https://cloud.google.com/run)
to deploy a web service that exposes a
[REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) to
get predictions from our trained model.

To run the web service, we configure Cloud Run to launch
[`gunicorn`](https://gunicorn.org)
on the container image we built.

We also provide environment variables with the Google Cloud project ID, a Cloud Storage path, a Compute Engine region, as well as the container image itself.
These are the Google Cloud resources we want the web service to use when launching new jobs or creating new resources.

Since calls to this web service could launch potentially expensive jobs in our project, we configure it to _only_ accept authenticated calls.

## 🏗️ Build container

We use Cloud Build to build the container image. After the image is built, it is available in Container Registry.

In [ ]:
# Set the Container Registry path for the sample container image.
app_container_image = f"gcr.io/{project}/geospatial-classification/app"

In [ ]:
# Build and push the container image.
# https://cloud.google.com/sdk/gcloud/reference/builds/submit
!gcloud builds submit serving_app \
  --tag="{app_container_image}" \
  --machine-type "e2-highcpu-8"

## 🐣 Deploy app

In [ ]:
# Deploy the web service to Cloud Run.
# https://cloud.google.com/sdk/gcloud/reference/run/deploy
!gcloud run deploy "geospatial-service" \
  --image="{app_container_image}" \
  --command="gunicorn" \
  --args="--threads=8,--timeout=0,main:app" \
  --region="us-central1" \
  --memory="1G" \
  --no-allow-unauthenticated

Now we need the web service URL to make calls to the REST API we just exposed. We can use `gcloud run services describe` to get the web service URL.

Since we only accept authorized calls in our web service, we also need to authenticate each call.
`gcloud` is already authenticated, so we can use `gcloud auth print-identity-token` to get quick access.

> ℹ️ For more information on how to do authenticated calls in Cloud Run, see the
> [Authentication overview](https://cloud.google.com/run/docs/authenticating/overview) page.

In [5]:
import subprocess

# Get the web service URL.
#   https://cloud.google.com/sdk/gcloud/reference/run/services/describe
service_url = subprocess.run(
    [ 'gcloud', 'run', 'services', 'describe', 'geospatial-service',
        f'--region={region}',
        f'--format=get(status.url)',
    ],
    capture_output=True,
).stdout.decode('utf-8').strip()
print(f"service_url: {service_url}")

# Get an identity token for authorized calls to our web service.
#   https://cloud.google.com/sdk/gcloud/reference/auth/print-identity-token
identity_token = subprocess.run(
    ['gcloud', 'auth', 'print-identity-token'],
    capture_output=True,
).stdout.decode('utf-8').strip()
print(f"identity_token: {identity_token}")

Finally, we can test that everything is working.

We included a `ping` method in our web service just to *make sure everything is working* as expected.
It simply returns back the arguments we passed to the call, as well as a response saying that the call was successful.

> 🛎️ This is a convenient way to make sure the web service is reachable, the authentication is working as expected, and the request arguments are passed correctly.

We can use Python's
[`requests`](https://docs.python-requests.org)
library.
The web service was built to always accept [JSON](https://www.w3schools.com/whatis/whatis_json.asp)-encoded requests, and returns JSON-encoded responses.

For a **request to be successful**, it **must**:

* Be an [`HTTP POST`](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST) request
* Contain the following **headers**:
  * `Authorization: Bearer IDENTITY_TOKEN`
  * `Content-Type: application/json`
* The data must be **valid JSON**, if *no arguments* are needed we can pass `{}` as an **empty object**.

For **ease of use**, `requests.post` has a
[`json` parameter](https://docs.python-requests.org/en/master/user/quickstart/#more-complicated-post-requests)
that **automatically attaches the header** `Content-Type: application/json` and encodes our data into a JSON string.

In [557]:
import requests

In [558]:
requests.post(
    url=f'{service_url}/ping',
    headers={'Authorization': f'Bearer {identity_token}'},
    json={'x': 42, 'message': 'Hello world!'},
).json()

{'args': 'Hello world!', 'response': 'Your request was successful! 🎉'}

# 4.🔮 Get Predictions

Now that we know our app is up and running, we can use it to make predictions.

Let's start by making a prediction for a particular power plant. To do this we will need to extract the Sentinel data from Earth Engine and send it in the body of the post requst to the prediction service.

We'll start with a plant located at the coordinates -84.80529, 39.11613, and then extract the satellite data from October 2021.

In [559]:
import json

In [560]:
plant_location = ee.Feature(ee.Geometry.Point([-84.80529, 39.11613]))

In [561]:
# Extract image data

def get_prediction_data(feature, start, end):
  '''Extracts Sentinel image as json at specific lat/lon and timestamp.'''

  image = ee.ImageCollection('COPERNICUS/S2').filterDate(start, end).select(BANDS).mosaic()

  fc = ee.FeatureCollection([
        image.neighborhoodToArray(ee.Kernel.square(16)).sampleRegions(
            collection=ee.FeatureCollection([feature]), scale=10).first()
    ])
  
  # download FeatureCollection as JSON
  url = fc.getDownloadURL('geojson')
  bytes = requests.get(url).content
  values=bytes.decode("utf-8")
  json_values = json.loads(values)
  return json_values['features'][0]['properties']

In [562]:
prediction_data = get_prediction_data(plant_location, '2021-10-01', '2021-10-31')

The prediction service **expects two things** the **input data** for the prediction as well as the Cloud Storage **path** where the model is stored.

In [563]:
requests.post(
    url=f'{service_url}/predict',
    headers={'Authorization': f'Bearer {identity_token}'},
    json={'data': prediction_data, 'bucket': bucket},
).json()

# 4. 🗺️ Visualize predictions

Let's visualize the results of a power plant in Spain. First, we get predictions for the four towers at this power plant.

In [533]:
locations = [ee.Feature(ee.Geometry.Point([-7.86444, 43.43717])),
             ee.Feature(ee.Geometry.Point([-7.86376, 43.43827])),
             ee.Feature(ee.Geometry.Point([-7.85755, 43.44075])),
             ee.Feature(ee.Geometry.Point([-7.85587, 43.44114])),]

In [534]:
plant_predictions = []
for location in locations:
    prediction_data = get_prediction_data(location, '2021-10-01', '2021-10-31')
    result = requests.post(
        url=f'{service_url}/predict',
        headers={'Authorization': f'Bearer {identity_token}'},
        json={'data': prediction_data, 'bucket': bucket},).json()
    preds.append(result['predictions']['predictions'][0][0][0][0])

Next, we can **plot** these points on a map. **Red** means our model predicts that the towers are **"on"**, and **blue** means that it predicts the towers are **"off"**.

In [522]:
import folium
import folium.plugins as folium_plugins
import branca.colormap as cm

In [523]:
lon = [-7.86444, -7.86376, -7.85755, -7.85587]
lat = [43.43717, 43.43827, 43.44075, 43.44114]

In [1]:
colormap = cm.LinearColormap(colors=['red','lightblue'], index=[90,100], vmin=90, vmax=100)
map = folium.Map([43.45, -7.87], zoom_start=14, tiles='CartoDB positron')
for loc, p in zip(zip(lat, lon), preds):
    folium.Circle(
        location=loc,
        radius=20,
        fill=True,
        color=colormap(p),
    ).add_to(map)

map.add_child(colormap)

display(map)

# 6. 🧹 Clean Up

To **avoid incurring charges** to your Google Cloud account for the resources used in this tutorial, either delete the project that contains the resources, or keep the project and delete the individual resources.

## Deleting the project

The **easiest** way to **eliminate billing** is to delete the project that you created for the tutorial.

To delete the project:

> ⚠️ Deleting a project has the following effects:
>
> * **Everything in the project is deleted.** If you used an existing project for this tutorial, when you delete it, you also delete any other work you've done in the project.
>
> * **Custom project IDs are lost.** When you created this project, you might have created a custom project ID that you want to use in the future. To preserve the URLs that use the project ID, such as an appspot.com URL, delete selected resources inside the project instead of deleting the whole project.
>
> If you plan to explore multiple tutorials and quickstarts, **reusing** projects can help you avoid exceeding project **quota limits**.

1. In the Cloud Console, go to the **Manage resources** page.

  <button>

  [Go to Manage resources](https://console.cloud.google.com/iam-admin/projects)

  </button>

1. In the project list, select the project that you want to delete, and then click **Delete**.

1. In the dialog, type the project ID, and then click **Shut down** to delete the project.